In [ ]:
!pip install torch==1.5.1
!pip install transformers==3.0.1
!pip install -U sentence-transformers
!pip install sentence-splitter
!pip install SentencePiece
import scipy
from sentence_transformers import SentenceTransformer

In [5]:
import json
f = open('') #change this to link any pegasus output file
data_pega = json.load(f)

In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def tokenize_data(sent_list):
  data = []
  # iterate through each sentence in the file
  for i in sent_list:
      temp = []
      # tokenize the sentence into words
      for j in word_tokenize(i):
          temp.append(j.lower())
      data.append(temp)
  return data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
def cand_ref(sent_list,text):
  refs = sent_list
  cands = [text] * len(refs)
  return cands,refs



>BERT_Score



In [ ]:
!pip install bert_score==0.3.4

In [9]:
from bert_score import BERTScorer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
import gc
def compute_bert_score(cands, refs):
  gc.collect()
  scorer = BERTScorer(lang="en")
  P, R, F1 = scorer.score(cands, refs)
  sent_score_bert = []
  for i in range(0,len(refs)):
    sent_score_bert.append((refs[i],F1.tolist()[i]))  
  del P
  del R
  del F1
  sent_score_bert.sort(key = lambda x: x[1])
  sent_score_bert.reverse()
  return sent_score_bert

In [11]:
def compute_semantic_similarity(data,sent_list):
  model = SentenceTransformer('bert-base-nli-mean-tokens')
  model.to(device)
  ref_embeddings = model.encode(sent_list)
  cand_embeddings = model.encode(data)
  distances = []
  for query, query_embedding in zip(sent_list, ref_embeddings):
      distance = scipy.spatial.distance.cdist([query_embedding], [cand_embeddings], "cosine")[0]
      distances.append((query,distance.tolist()[0]))
  distances.sort(key = lambda x: x[1])
  return distances



> SELF-BLEU



In [16]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import numpy as np
import copy

def get_bleu_score(sentence, remaining_sentences):
    lst = []
    smoothie = SmoothingFunction().method4
    # for i in remaining_sentences:
    bleu = sentence_bleu(remaining_sentences, sentence, smoothing_function=smoothie)
    # lst.append(bleu)
    return bleu


def calculate_selfBleu(sentences):
    '''
    sentences - list of sentences generated by NLG system
    '''
    bleu_scores = []
	
    for i in sentences:
        sentences_copy = copy.deepcopy(sentences)
        remaining_sentences = sentences_copy.remove(i)
        bleu = get_bleu_score(i,sentences_copy)
        bleu_scores.append(bleu)

    return [np.mean(bs) for bs in bleu_scores]

In [17]:
def compute_self_bleu(data,sent_list): 
  scores = calculate_selfBleu(data)
  sent_score = []
  for i in range(0,len(sent_list)):
    sent_score.append((sent_list[i],scores[i]))
  sent_score.sort(key = lambda x: x[1])
  return sent_score

In [18]:
# with open('/content/drive/MyDrive/'+c+'/self_bleu.txt', 'w+') as f:
#   for items in sent_score:
#     f.write('%s\n' %str(items))
#   print("File written successfully")
# f.close()



> Levenstein



In [19]:
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 36.9 MB/s 
     |████████████████████████████████| 2.2 MB 52.6 MB/s 


In [20]:
import Levenshtein

In [21]:
import numpy as np
import copy

def get_Levenshtein(sentence, remaining_sentences):
    lst = []
    for i in remaining_sentences:
        lev = Levenshtein.distance(sentence, i)
        lst.append(lev)
    return lst


def calculate_selfLevenshtein(sentences):
    '''
    sentences - list of sentences generated by NLG system
    '''
    lev_scores = []
	
    for i in sentences:
        sentences_copy = copy.deepcopy(sentences)
        remaining_sentences = sentences_copy.remove(i)
        lev = get_Levenshtein(i,sentences_copy)
        lev_scores.append(lev)

    return [np.mean(ls) for ls in lev_scores]

In [22]:
def compute_selfLevenshtein(sent_list):
  scores = calculate_selfLevenshtein(sent_list)
  sent_score_new = []
  for i in range(0,len(sent_list)):
   sent_score_new.append((sent_list[i],scores[i]))
  sent_score_new.sort(key = lambda x: x[1])
  sent_score_new.reverse()
  return sent_score_new



> avg_index



In [24]:
def compute_avg_index(sent_score_bert,sent_score_new,sent_list):
  #for context
  sent_score_bert_sent = []
  for i in sent_score_bert:
    sent_score_bert_sent.append(i[0])
  #for diversity
  sent_score_sent = []
  for i in sent_score_new:
    sent_score_sent.append(i[0])
  avg_index = []
  for i in sent_list:
    avg_index.append((i,sent_score_bert_sent.index(i)+sent_score_sent.index(i)/2))
  avg_index.sort(key = lambda x: x[1])
  return avg_index



> greedy selection



In [26]:
def get_bleu_score_greedy(sentence, remaining_sentences):
    smoothie = SmoothingFunction().method4
    lst = []
    for i in remaining_sentences:
        bleu = sentence_bleu(sentence, i, smoothing_function=smoothie, weights = (0.1,0.1,0.1,0.7))
        lst.append(bleu)
    return lst

In [27]:
def compute_final(avg_index):
  texts = []
  texts.append(avg_index[0][0])
  for j in range(0,4):
    avg_index_sent = []
    for i in avg_index[1:25]:
      scores_final = get_Levenshtein(i[0],texts)
      avg_index_sent.append((i[0],np.mean(scores_final)))
    avg_index_sent.sort(key = lambda x: x[1])
    avg_index_sent.reverse()
    for i in avg_index_sent:
      if i[0] not in texts:
        texts.append(i[0])
        break
  return texts

In [28]:
d = {}
count = 1

In [ ]:
# import gc
# import time
for key, value in data_pega.items():
  count += 1
  text = key
  sent_list = value
  data = tokenize_data(sent_list)
  cands, refs = cand_ref(sent_list,text)
  sent_score_bert = compute_semantic_similarity(text, sent_list)
  sent_score_new = compute_selfLevenshtein(sent_list)
  avg_index = compute_avg_index(sent_score_bert,sent_score_new,sent_list)
  final_texts = compute_final(avg_index)
  d[key] = final_texts
  # del sent_score_bert

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
json_object = json.dumps(d, indent = 0)
with open('', 'w') as outfile: #output file to check
    outfile.write(json_object)